In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_69168/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [5]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
1,model_14_7_5,0.998562,0.995669,0.999219,0.998009,0.173452,0.385403,0.147853,0.273614,"Hidden Size=[35], regularizer=0.1, learning_ra..."
3,model_14_7_4,0.998522,0.996532,0.999370,0.998402,0.178254,0.308660,0.119291,0.219546,"Hidden Size=[35], regularizer=0.1, learning_ra..."
6,model_14_7_3,0.998435,0.997358,0.999515,0.998779,0.188717,0.235128,0.091869,0.167712,"Hidden Size=[35], regularizer=0.1, learning_ra..."
9,model_14_7_2,0.998286,0.998112,0.999647,0.999124,0.206691,0.168017,0.066753,0.120364,"Hidden Size=[35], regularizer=0.1, learning_ra..."
14,model_14_7_1,0.998055,0.998746,0.999760,0.999414,0.234527,0.111588,0.045490,0.080482,"Hidden Size=[35], regularizer=0.1, learning_ra..."
26,model_17_9_10,0.997720,0.995590,0.992486,0.994411,0.274961,0.530605,0.739285,0.628806,"Hidden Size=[45], regularizer=0.01, learning_r..."
28,model_14_7_0,0.997718,0.999198,0.999841,0.999622,0.275194,0.071407,0.030107,0.051971,"Hidden Size=[35], regularizer=0.1, learning_ra..."
32,model_17_9_9,0.997661,0.995824,0.993058,0.994779,0.282123,0.502452,0.683047,0.587438,"Hidden Size=[45], regularizer=0.01, learning_r..."
40,model_14_6_9,0.997607,0.994691,0.994576,0.994730,0.288615,0.621971,0.681834,0.650141,"Hidden Size=[35], regularizer=0.1, learning_ra..."
43,model_17_9_8,0.997567,0.996059,0.993662,0.995161,0.293429,0.474206,0.623544,0.544481,"Hidden Size=[45], regularizer=0.01, learning_r..."
